In [ ]:
from notebook_utils import *

import imageio

import random
import os
import time

%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

# Segmentation / classification report

In [ ]:
import clb
from clb.dataprep.utils import VolumeIter
from clb.stats import cells
import pandas

In [ ]:
# parameters
input_file = None
    #r"D:\Fafa\MIT\CellDx\general_dump\ce_class_final_tylko_8\results\classify_test\classes_test_input.tif"
input_series = None
segmentation_file = None
    #r"D:\Fafa\MIT\CellDx\general_dump\ce_class_final_tylko_8\results\classify_test\classes_test_segment.tif"
classification_ki67_file = None 
    #r"D:\Fafa\MIT\CellDx\general_dump\ce_class_final_tylko_8\results\classify_test\classes_test_ki67_output.tif"
classification_epith_file = None
    #r"D:\Fafa\MIT\CellDx\general_dump\ce_class_final_tylko_8\results\classify_test\classes_test_epith_output.tif"

In [ ]:
# information about report
input_path_series = os.path.basename(input_file) 
if input_series is not None:
    input_path_series += " series: " + str(input_series)
else:
    input_series = 0
printmd("## File: {0} ({1}): \n&emsp;{2}"
        .format(input_path_series, time.ctime(os.path.getmtime(input_file)), input_file))

printmd("### Segmentation ({1}): \n&emsp;{0}"
        .format(segmentation_file, time.ctime(os.path.getmtime(segmentation_file))))
if classification_ki67_file is not None:
    printmd("### Classification Ki67 ({1}): \n&emsp;{0}"
            .format(classification_ki67_file, time.ctime(os.path.getmtime(classification_ki67_file))))
if classification_epith_file is not None:
    printmd("### Classification Epithelium ({1}): \n&emsp;{0}"
            .format(classification_epith_file, time.ctime(os.path.getmtime(classification_epith_file))))

# Data results samples

In [ ]:
# load data and check if classification
from clb.dataprep.utils import rescale_to_float
from clb.utils import overlay_sample_boundaries

input_data = VolumeIter(input_file, series=input_series)
slices_num = len(input_data)
white_on_input = 255 if input_data[0].ndim == 2 else (255,255,255)

printmd("&emsp;**Data resolution:** X={0}, Y={1}".format(input_data[0].shape[1], input_data[0].shape[0]))
printmd("&emsp;**Data depth:** ", slices_num)

segment_data = VolumeIter(segmentation_file)
# TODO use actuall results instead of trying to reinterpret visualisation 
if classification_ki67_file is not None:
    class_ki67_data =  VolumeIter(classification_ki67_file)
    rescaled_ki67 = rescale_to_float(np.array(class_ki67_data), float_type='float16')
    thresholded_ki67 = rescaled_ki67 > 0.5
if classification_epith_file is not None:
    class_epith_data =  VolumeIter(classification_epith_file)
    rescaled_epith = rescale_to_float(np.array(class_epith_data), float_type='float16')
    thresholded_epith = rescaled_epith > 0.5
    
slices_titles = ["Slice at 0 depth", "Slice at 1/3 depth", "Slice at 2/3 depth"]

In [ ]:
printmd("### Input imagery")
show_all(1,3, input_data[0], input_data[1 * slices_num//3], input_data[2 * slices_num//3], scale=30, titles=slices_titles)

In [ ]:
printmd("### Segmentation")
show_all(1,3, segment_data[0], segment_data[1 * slices_num//3], segment_data[2 * slices_num//3], scale=30, cmap='rand', titles=slices_titles)

In [ ]:
fig = None
if classification_ki67_file is not None:
    printmd("### Classification Ki67")
    fig = show_all(1,3, 
                   overlay_sample_boundaries(input_data, thresholded_ki67, 0, colour=white_on_input),
                   overlay_sample_boundaries(input_data, thresholded_ki67, 1 * slices_num//3, colour=white_on_input),
                   overlay_sample_boundaries(input_data, thresholded_ki67, 2 * slices_num//3, colour=white_on_input),
                   scale=30, titles=slices_titles)
fig

In [ ]:
fig = None
if classification_epith_file is not None:
    printmd("### Classification Epithelium")
    fig = show_all(1,3, 
                   overlay_sample_boundaries(input_data, thresholded_epith, 0, colour=white_on_input),
                   overlay_sample_boundaries(input_data, thresholded_epith, 1 * slices_num//3, colour=white_on_input),
                   overlay_sample_boundaries(input_data, thresholded_epith, 2 * slices_num//3, colour=white_on_input),
                   scale=30, titles=slices_titles)
fig

# Statistics

In [ ]:
loaded_segment = np.array(segment_data)
stats = cells.cell_stats(loaded_segment)

if classification_ki67_file is not None:
    stats_ki67 = cells.cell_stats(loaded_segment, rescaled_ki67)
    
if classification_epith_file is not None:
    stats_epith = cells.cell_stats(loaded_segment, rescaled_epith)

In [ ]:
printmd("**Cell count:** ", stats['cell_number'])
if classification_ki67_file is not None:
    printmd("**Ki67 cell count:** ", stats_ki67['class_cell_number'])
if classification_epith_file is not None:
    printmd("**Epithelium cell count:** ", stats_epith['class_cell_number'])

In [ ]:
slice_data = pandas.DataFrame(list(stats['slices']))
if classification_ki67_file:
    slice_data = slice_data.join(pandas.DataFrame(list(stats_ki67['slices'])).add_suffix('_ki67'))
if classification_epith_file:
    slice_data = slice_data.join(pandas.DataFrame(list(stats_epith['slices'])).add_suffix('_epith'))

In [ ]:
ax = slice_data.plot(title='Cell number change though stack', y='cell_number', figsize=(15, 6), grid=True)
ax.set_ylim(ymin=0)

In [ ]:
if classification_ki67_file or classification_epith_file:
    ys = []
    if classification_ki67_file:
        ys += ['class_cell_number_ki67']
    if classification_epith_file:
        ys += ['class_cell_number_epith']
    slice_data.plot(title='Class cell number change though stack', 
                y=ys, figsize=(15, 8), grid=True);
    ax.set_ylim(ymin=0)